# Notebook for generating

# Import

In [1]:
import os
import random
import calendar
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Literal
from IPython.display import display

# Settings & Utils

## Constants & Helper

In [2]:
RESET_DB = False

UPDATE_TRANSACTION_TABLES_THIS_RUN = True

ADD_RECORDS_THIS_RUN = True
UPDATE_RECORDS_THIS_RUN = True
DELETE_RECORDS_THIS_RUN = True

In [3]:
SEPERATOR = "|"
CHANGE_FIELD = "change_field"
TRANSACTION_DATE_COLUMN = "transaction_date"
START_TRANSACTION_DATE = datetime(2000, 1, 1)

TBL_EMPLOYEE = "employee"
TBL_CLIENT = "client"
TBL_BUSINESSPARTNER = "businesspartner"
TBL_COSTCENTER = "costcenter"
TBL_DEPARTMENT = "department"
TBL_TASK = "task"
TBL_PROJECT = "project"

TBL_USER = "user"
TBL_PROJECT = "project"
TBL_PROJECTTIME = "projecttime"
TBL_PAY_TYPE = "pay_type"
TBL_EMPLOYEE_PAY = "employee_pay"

DB_ATOSS = "atoss"
DB_PCT = "pct"
DB_DATEV = "datev"

SCHEMA_DBO = "dbo"
SCHEMA_MDM = "mdm"

In [4]:
def create_wide_df(nrows: int, ncolumns_each: int = 50) -> pd.DataFrame:
    row_rng = range(0, nrows)
    col_rng = range(0, ncolumns_each)
    string_values = [
        [f"string-value ({row_idx + 1}, {col_idx + 1})" for col_idx in col_rng]
        for row_idx in row_rng
    ]
    random_num_values = np.random.randint(0, 10_000_000, size=(nrows, ncolumns_each))

    date_values = [
        [datetime(2005, 5, 5, 5, 5, 5, 500) for _ in col_rng]
        for _ in row_rng
    ]

    column_names = [f"bdcolumn{i}" for i in range(1, (3 * ncolumns_each) + 1)]

    df_string_values = pd.DataFrame(string_values)
    df_num_values = pd.DataFrame(random_num_values)
    df_date_values = pd.DataFrame(date_values)

    df = pd.concat([df_string_values, df_num_values, df_date_values], axis=1)
    df.columns = column_names

    return df

def generate_wide_df(df: pd.DataFrame, ncolumns_each: int = 50) -> pd.DataFrame:
    df_bd = create_wide_df(nrows=df.shape[0], ncolumns_each=ncolumns_each)
    return pd.concat([df, df_bd], axis=1)

In [5]:
@dataclass(frozen=True)
class MdTable:
    tablename: str

    @property
    def tablepath(self) -> str:
        return f"./_md_{self.tablename}.csv"
    
    def read(self, sep: str = SEPERATOR, **kwargs) -> pd.DataFrame:
        return pd.read_csv(self.tablepath, sep=sep, **kwargs)


@dataclass(frozen=True)
class DbTable:
    db: str
    schema: str
    tablename: str

    @property
    def table_total_name(self) -> str:
        return f"{self.db}.{self.schema}.{self.tablename}"
    
    @property
    def tablepath(self) -> str:
        return f"./{self.table_total_name}.csv"
    
    @property
    def is_existing(self) -> bool:
        return os.path.exists(self.tablepath)
    
    def save(
            self,
            df: pd.DataFrame,
            sep: str = SEPERATOR,
            mode: Literal["a", "w"] = "w"
        ) -> None:
        df.to_csv(
            self.tablepath,
            sep=sep,
            index=False,
            encoding="UTF-8",
            mode=mode,
            header=(mode == "w")
        )
        print(f"[{str(datetime.now())[:19]}] TABLE successfully saved at: {self.tablepath}   SHAPE: {df.shape}")
        
    def read(self, sep: str = SEPERATOR, columns: list[str] = None, **kwargs) -> pd.DataFrame:
        return pd.read_csv(self.tablepath, sep=sep, usecols=columns, **kwargs)
    
    def delete(self) -> None:
        if not self.is_existing:
            log = f"{self.table_total_name} not existing."
            print(log)
            return
        
        os.remove(self.tablepath)
        log = f"TABLE {self.table_total_name} successfully removed."
        print(log)

def get_update_count(row: str) -> str:
    split_str = " uc: "
    if split_str in row:
        value, uc = row.split(split_str)
        uc = int(uc) + 1
        return f"{value} uc: {uc}"
    return f"{row} uc: 1"

In [6]:
def pd_crossjoin(
        df: pd.DataFrame,
        join_table: DbTable,
        id_column_name_jt: str,
        id_new_column_name: str,
        include_columns: list[str] = None
        ) -> pd.DataFrame:
    if include_columns is None:
        include_columns = []
        
    include_columns = [id_column_name_jt] + include_columns
    df_join = join_table.read()[include_columns] \
        .rename(columns={id_column_name_jt: id_new_column_name})

    return pd.merge(df, df_join, how="cross").drop_duplicates()

In [7]:
def pd_add_rows(
        *,
        df: pd.DataFrame,
        df_md: pd.DataFrame,
        nrows: int,
        id_columns: list[str] = None,
        crossjoin_tables: DbTable = None
    ) -> pd.DataFrame:
    if id_columns is None:
        id_columns = ["id"]
    
    df_current_ids = df[id_columns]

    added_filter = (df_md[id_columns[0]].isin(df_current_ids[id_columns[0]]))
    for i in range(1, len(id_columns)):
        added_filter &= (df[id_columns[i]].isin(df_current_ids[id_columns[i]]))

    df_new_records = df_md[~added_filter].sample(nrows)
    if crossjoin_tables is not None:
        for i in range(1, len(id_columns)):
            df_new_records = pd_crossjoin(
                df=df_new_records,
                join_table=crossjoin_tables[i - 1],
                id_column_name_jt=id_columns[0],
                id_new_column_name=id_columns[i]
            )

    df_new_records = df_new_records.sample(nrows)

    print(f"New Records (nrows={nrows}):")
    display(df_new_records)

    return pd.concat([df, df_new_records], axis=0).reset_index(drop=True)


def pd_update_rows(
        *,
        df: pd.DataFrame,
        nrows: int,
        id_columns: list = None,
        cf_column: str = CHANGE_FIELD
    ) -> pd.DataFrame:
    if id_columns is None:
        id_columns = ["id"]

    df_sample_ids = df.sample(nrows)[id_columns]

    update_filter = (df[id_columns[0]].isin(df_sample_ids[id_columns[0]]))
    for i in range(1, len(id_columns)):
        update_filter &= (df[id_columns[i]].isin(df_sample_ids[id_columns[i]]))

    df.loc[update_filter, cf_column] = df.loc[update_filter, cf_column] \
                                            .apply(get_update_count)
    print(f"Updated Records (nrows={nrows}):")
    display(df[update_filter])

    return df.reset_index(drop=True)


def pd_delete_rows(*, df: pd.DataFrame, nrows: int) -> pd.DataFrame:
    # TODO: implement multiple column id
    delete_ids = list(df.sample(nrows)["id"])
    delete_filter = df["id"].isin(delete_ids)

    print(f"Removed Records (nrows={nrows}):")
    display(df[delete_filter])

    return df[~delete_filter].reset_index(drop=True)

In [8]:
def get_end_of_month(dt: datetime, offset_months: int = 0):
    dt = datetime(dt.year, dt.month, 1) + timedelta(days=(offset_months * 31))
    _, last_day = calendar.monthrange(dt.year, dt.month)
    return datetime(dt.year, dt.month, last_day)

def get_next_transaction_datetime(
        table: DbTable,
        sep: str = SEPERATOR,
        date_column: str = TRANSACTION_DATE_COLUMN,
        default_date: datetime = START_TRANSACTION_DATE
    ) -> datetime:
    try:
        date_str = pd.read_csv(
            table.tablepath,
            sep=sep,
            usecols=[date_column]
        ).max().values[0]
        next_transaction_dt = datetime.strptime(date_str, '%Y-%m-%d') + timedelta()
        return get_end_of_month(next_transaction_dt, offset_days=1)
    except:
        return get_end_of_month(default_date)

## Table Setup

### MasterData Tables

In [9]:
tbl_md_employee = MdTable(TBL_EMPLOYEE)
tbl_md_client = MdTable(TBL_CLIENT)
tbl_md_businesspartner = MdTable(TBL_BUSINESSPARTNER)
tbl_md_costcenter = MdTable(TBL_COSTCENTER)
tbl_md_department = MdTable(TBL_DEPARTMENT)
tbl_md_project = MdTable(TBL_PROJECT)
tbl_md_task = MdTable(TBL_TASK)

display(
    tbl_md_employee,
    tbl_md_client,
    tbl_md_businesspartner,
    tbl_md_costcenter,
    tbl_md_department,
    tbl_md_project,
    tbl_md_task
)

MdTable(tablename='employee')

MdTable(tablename='client')

MdTable(tablename='businesspartner')

MdTable(tablename='costcenter')

MdTable(tablename='department')

MdTable(tablename='project')

MdTable(tablename='task')

In [10]:
df_md_employee = tbl_md_employee.read(parse_dates=["leave_date"])
df_md_client = tbl_md_client.read()
df_md_businesspartner = tbl_md_businesspartner.read()
df_md_costcenter = tbl_md_costcenter.read()
df_md_department = tbl_md_department.read()
df_md_project = tbl_md_project.read()
df_md_task = tbl_md_task.read()

### DB Tables

In [11]:
tbl_db_pct_users = DbTable(DB_PCT, SCHEMA_DBO, TBL_USER)
tbl_db_pct_project = DbTable(DB_PCT, SCHEMA_DBO, TBL_PROJECT)
tbl_db_pct_department = DbTable(DB_PCT, SCHEMA_DBO, TBL_DEPARTMENT)
tbl_db_pct_task = DbTable(DB_PCT, SCHEMA_DBO, TBL_TASK)
tbl_db_pct_businesspartner = DbTable(DB_PCT, SCHEMA_DBO, TBL_BUSINESSPARTNER)
tbl_db_pct_projecttime = DbTable(DB_PCT, SCHEMA_DBO, TBL_PROJECTTIME)
tbl_db_pct_employee_mdm = DbTable(DB_PCT, SCHEMA_MDM, TBL_EMPLOYEE)

tbl_db_datev_employee = DbTable(DB_DATEV, SCHEMA_DBO, TBL_EMPLOYEE)
tbl_db_datev_client = DbTable(DB_DATEV, SCHEMA_DBO, TBL_CLIENT)
tbl_db_datev_department = DbTable(DB_DATEV, SCHEMA_DBO, TBL_DEPARTMENT)
tbl_db_datev_costcenter = DbTable(DB_DATEV, SCHEMA_DBO, TBL_COSTCENTER)
tbl_db_datev_paytype = DbTable(DB_DATEV, SCHEMA_DBO, TBL_PAY_TYPE)
tbl_db_datev_employee_pay = DbTable(DB_DATEV, SCHEMA_DBO, TBL_EMPLOYEE_PAY)

ALL_TABLES = [
    tbl_db_pct_users,
    tbl_db_pct_project,
    tbl_db_pct_department,
    tbl_db_pct_task,
    tbl_db_pct_businesspartner,
    tbl_db_pct_projecttime,
    tbl_db_pct_employee_mdm,
    tbl_db_datev_employee,
    tbl_db_datev_client,
    tbl_db_datev_department,
    tbl_db_datev_costcenter,
    tbl_db_datev_paytype,
    tbl_db_datev_employee_pay
]

display(
    tbl_db_pct_users,
    tbl_db_pct_project,
    tbl_db_pct_department,
    tbl_db_pct_task,
    tbl_db_pct_businesspartner,
    tbl_db_pct_projecttime,
    tbl_db_pct_employee_mdm,
    tbl_db_datev_employee,
    tbl_db_datev_client,
    tbl_db_datev_department,
    tbl_db_datev_costcenter,
    tbl_db_datev_paytype,
    tbl_db_datev_employee_pay
)

DbTable(db='pct', schema='dbo', tablename='user')

DbTable(db='pct', schema='dbo', tablename='project')

DbTable(db='pct', schema='dbo', tablename='department')

DbTable(db='pct', schema='dbo', tablename='task')

DbTable(db='pct', schema='dbo', tablename='businesspartner')

...

DbTable(db='datev', schema='dbo', tablename='client')

DbTable(db='datev', schema='dbo', tablename='department')

DbTable(db='datev', schema='dbo', tablename='costcenter')

DbTable(db='datev', schema='dbo', tablename='pay_type')

DbTable(db='datev', schema='dbo', tablename='employee_pay')

# Generate or Update databasefiles

## Reset DB if needed

In [12]:
if RESET_DB:
    table: DbTable
    for table in ALL_TABLES:
        table.delete()
else:
    print("No Reset!")

No Reset!


## ERP-System - datev

### dbo.clients

#### Create if not existing

In [13]:
INIT_CLIENT_COUNT = 10

if not tbl_db_datev_client.is_existing:
    df_init = df_md_client.sample(INIT_CLIENT_COUNT)
    tbl_db_datev_client.save(df_init)

df_db_datev_client = tbl_db_datev_client.read()
df_db_datev_client.head()

[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.client.csv   SHAPE: (10, 4)


,id,name,address,change_field
0,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf
1,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf
2,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf
3,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf
4,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf


#### Update Records

In [14]:
update_record_count = random.choice(range(0, 4))

df_db_datev_client = pd_update_rows(
    df=df_db_datev_client,
    nrows=update_record_count
)

tbl_db_datev_client.save(df_db_datev_client)

Updated Records (nrows=1):


,id,name,address,change_field
9,3,Apex Corporation GmbH,"Musterstraße 3, 10109 Berlin",cf uc: 1


[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.client.csv   SHAPE: (10, 4)


#### Add Records

In [15]:
add_record_count = random.choice(range(0, 2)) if ADD_RECORDS_THIS_RUN else 0

df_db_datev_client = pd_add_rows(
    df=df_db_datev_client,
    df_md=df_md_client,
    nrows=add_record_count
)

tbl_db_datev_client.save(df_db_datev_client)

New Records (nrows=0):


,id,name,address,change_field


[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.client.csv   SHAPE: (10, 4)


### dbo.department

#### Create if not existing

In [16]:
INIT_DEPARTMENT_COUNT = 5

if not tbl_db_datev_department.is_existing:
    df_init = df_md_department.sample(INIT_DEPARTMENT_COUNT)

    df_init = pd_crossjoin(
        df=df_init,
        join_table=tbl_db_datev_client,
        id_column_name_jt="id",
        id_new_column_name="client_id"
    )

    tbl_db_datev_department.save(df_init)

df_db_datev_department = tbl_db_datev_department.read()
df_db_datev_department.sample(5).head()

[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.department.csv   SHAPE: (50, 4)


,id,name,change_field,client_id
29,47,Sustainability,cf,3
30,31,Market Research,cf,57
9,27,Internal Audit,cf,3
26,47,Sustainability,cf,49
31,31,Market Research,cf,19


#### Update Records

In [17]:
update_record_count = random.choice(range(0, 4))

df_db_datev_department = pd_update_rows(
    df=df_db_datev_department,
    nrows=update_record_count,
    id_columns=["id", "client_id"]
)

tbl_db_datev_department.save(df_db_datev_department)

Updated Records (nrows=1):


,id,name,change_field,client_id
46,8,Corporate Strategy,cf uc: 1,49


[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.department.csv   SHAPE: (50, 4)


#### Add Records

In [18]:
add_record_count = random.choice(range(0, 2)) if ADD_RECORDS_THIS_RUN else 0

df_db_datev_department = pd_add_rows(
    df=df_db_datev_department,
    df_md=df_md_department,
    nrows=add_record_count,
    id_columns=["id", "client_id"],
    crossjoin_tables=[tbl_db_datev_client]
)

tbl_db_datev_department.save(df_db_datev_department)

New Records (nrows=1):


,id,name,change_field,client_id
4,15,Employee Relations,cf,17


[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.department.csv   SHAPE: (51, 4)


### dbo.costcenter

#### Create if not existing

In [19]:
INIT_COSTCENTER_COUNT = 20

if not tbl_db_datev_costcenter.is_existing:
    df_init = df_md_costcenter.sample(INIT_COSTCENTER_COUNT)

    df_init = pd_crossjoin(
        df=df_init,
        join_table=tbl_db_datev_department,
        id_column_name_jt="id",
        id_new_column_name="department_id"
    )

    df_init = pd_crossjoin(
        df=df_init,
        join_table=tbl_db_datev_client,
        id_column_name_jt="id",
        id_new_column_name="client_id"
    )

    tbl_db_datev_costcenter.save(df_init)


df_db_datev_costcenter = tbl_db_datev_costcenter.read()
df_db_datev_costcenter.sample(5).head()

[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.costcenter.csv   SHAPE: (1200, 7)


,id,buKr,costcenter_short,name,change_field,department_id,client_id
650,1942,1748,33650,KSTNAME1942,cf,15,57
1141,148,1736,10550,KSTNAME148,cf,27,19
534,948,1741,49050,KSTNAME948,cf,15,17
911,1569,1745,69400,KSTNAME1569,cf,5,19
1195,148,1736,10550,KSTNAME148,cf,15,50


In [20]:
unique_together_count = \
    len(df_db_datev_costcenter["id"].unique()) * \
    len(df_db_datev_costcenter["client_id"].unique()) * \
    len(df_db_datev_costcenter["department_id"].unique())

unique_together_count, df_db_datev_costcenter.shape[0], df_db_datev_costcenter.drop_duplicates().shape[0]

(1200, 1200, 1200)

#### Update Records

In [21]:
update_record_count = random.choice(range(0, 7))

df_db_datev_costcenter = pd_update_rows(
    df=df_db_datev_costcenter,
    nrows=update_record_count,
    id_columns=["id", "client_id", "department_id"]
)

tbl_db_datev_costcenter.save(df_db_datev_costcenter)

Updated Records (nrows=3):


,id,buKr,costcenter_short,name,change_field,department_id,client_id
745,14,1735,17150,KSTNAME14,cf uc: 1,47,50
747,14,1735,17150,KSTNAME14,cf uc: 1,47,92
765,14,1735,17150,KSTNAME14,cf uc: 1,8,50
767,14,1735,17150,KSTNAME14,cf uc: 1,8,92
775,14,1735,17150,KSTNAME14,cf uc: 1,15,50
777,14,1735,17150,KSTNAME14,cf uc: 1,15,92
865,936,1741,42450,KSTNAME936,cf uc: 1,47,50
867,936,1741,42450,KSTNAME936,cf uc: 1,47,92
885,936,1741,42450,KSTNAME936,cf uc: 1,8,50
887,936,1741,42450,KSTNAME936,cf uc: 1,8,92


[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.costcenter.csv   SHAPE: (1200, 7)


#### Add Records

In [22]:
add_record_count = random.choice(range(0, 3)) if ADD_RECORDS_THIS_RUN else 0

df_db_datev_costcenter = pd_add_rows(
    df=df_db_datev_costcenter,
    df_md=df_md_costcenter,
    nrows=add_record_count,
    id_columns=["id", "client_id", "department_id"],
    crossjoin_tables=[tbl_db_datev_client, tbl_db_datev_department]
)

tbl_db_datev_costcenter.save(df_db_datev_costcenter)

New Records (nrows=0):


,id,buKr,costcenter_short,name,change_field,client_id,department_id


[2023-12-21 22:59:50] TABLE successfully saved at: ./datev.dbo.costcenter.csv   SHAPE: (1200, 7)


### dbo.employee

#### Create if not existing

In [23]:
INIT_EMPLOYEE_COUNT = 5000
INIT_EMPLOYEE_TOTAL_COUNT = INIT_EMPLOYEE_COUNT * 6

if not tbl_db_datev_employee.is_existing:
    df_init = df_md_employee.sample(INIT_EMPLOYEE_COUNT)

    df_init = pd_crossjoin(
        df=df_init,
        join_table=tbl_db_datev_client,
        id_column_name_jt="id",
        id_new_column_name="client_id"
    )

    df_init = pd_crossjoin(
        df=df_init,
        join_table=tbl_db_datev_department,
        id_column_name_jt="id",
        id_new_column_name="costcenter_id"
    )

    df_init = df_init.sample(INIT_EMPLOYEE_TOTAL_COUNT)

    tbl_db_datev_employee.save(df_init)

df_db_datev_employee = tbl_db_datev_employee.read()
df_db_datev_employee.head()

[2023-12-21 22:59:51] TABLE successfully saved at: ./datev.dbo.employee.csv   SHAPE: (30000, 10)


,id,firstname,lastname,birthdate,entry_date,salary,leave_date,change_field,client_id,costcenter_id
0,3769,Avery,Harris,1985-08-07,2024-02-03,6111.82,NaN,cf,92,8
1,7444,Osman,Ekinci,1951-04-13,2003-10-30,6103.96,NaN,cf,57,8
2,1060,Clara,Castillo,1981-02-26,2024-07-31,4865.90,NaN,cf,50,27
3,5086,Sofia,Novotná,1977-12-31,2015-01-19,4884.41,NaN,cf,3,47
4,8159,Julien,Lefevre,1989-12-09,1997-08-19,6452.67,NaN,cf,17,47


#### Update Records

In [24]:
update_record_count = random.choice(range(0, 7))

df_db_datev_employee = pd_update_rows(
    df=df_db_datev_employee,
    nrows=update_record_count
)

tbl_db_datev_employee.save(df_db_datev_employee)

Updated Records (nrows=4):


,id,firstname,lastname,birthdate,entry_date,salary,leave_date,change_field,client_id,costcenter_id
1747,3195,Matthew,Lee,1996-06-22,2022-07-07,4655.44,NaN,cf uc: 1,92,31
2462,5508,Maximilian,Wagner,1974-01-09,2011-06-19,6348.25,NaN,cf uc: 1,50,5
2685,5508,Maximilian,Wagner,1974-01-09,2011-06-19,6348.25,NaN,cf uc: 1,92,47
2926,5508,Maximilian,Wagner,1974-01-09,2011-06-19,6348.25,NaN,cf uc: 1,3,31
5972,3618,Eva,Müller,2002-10-17,2017-02-07,5009.48,NaN,cf uc: 1,3,31
8631,3618,Eva,Müller,2002-10-17,2017-02-07,5009.48,NaN,cf uc: 1,91,47
10790,1414,Irene,Adler,2003-10-06,2000-04-07,5118.01,NaN,cf uc: 1,91,5
11484,3195,Matthew,Lee,1996-06-22,2022-07-07,4655.44,NaN,cf uc: 1,66,15
11643,3618,Eva,Müller,2002-10-17,2017-02-07,5009.48,NaN,cf uc: 1,19,31
11746,1414,Irene,Adler,2003-10-06,2000-04-07,5118.01,NaN,cf uc: 1,57,27


[2023-12-21 22:59:51] TABLE successfully saved at: ./datev.dbo.employee.csv   SHAPE: (30000, 10)


#### Add Records

In [25]:
add_record_count = random.choice(range(2, 20)) if ADD_RECORDS_THIS_RUN else 0

df_db_datev_employee = pd_add_rows(
    df=df_db_datev_employee,
    df_md=df_md_employee,
    nrows=add_record_count,
    crossjoin_tables=[tbl_db_datev_client, tbl_db_datev_department],
    id_columns=["id", "client_id", "costcenter_id"]
)

tbl_db_datev_employee.save(df_db_datev_employee)

New Records (nrows=17):


,id,firstname,lastname,birthdate,entry_date,salary,leave_date,change_field,client_id,costcenter_id
2907,4510,William,Thomas,1950-02-03,2013-07-15,4679.36,NaT,cf,92,27
5712,1788,Lily,Robinson,1963-11-09,2003-03-14,5954.94,NaT,cf,66,27
3549,7252,Maximilian,Wagner,1961-09-02,2011-01-17,5520.88,NaT,cf,3,31
4182,2012,Luigi,Ferrari,1972-06-02,2006-05-28,4813.31,NaT,cf,66,27
5089,1280,Maja,Kwiatkowska,1998-11-20,2000-05-25,4637.82,NaT,cf,3,8
6181,3147,Ella,Lee,1985-04-19,2024-04-04,5256.41,NaT,cf,19,5
4793,1280,Maja,Kwiatkowska,1998-11-20,2000-05-25,4637.82,NaT,cf,30,15
8578,2833,Liam,Johnson,1980-04-02,2008-11-19,5772.65,NaT,cf,91,5
4997,1280,Maja,Kwiatkowska,1998-11-20,2000-05-25,4637.82,NaT,cf,92,15
2192,7297,Luca,Fischer,1999-02-04,2011-11-27,5106.68,NaT,cf,66,15


[2023-12-21 22:59:51] TABLE successfully saved at: ./datev.dbo.employee.csv   SHAPE: (30017, 10)


### dbo.paytype [Frozen Master Data Table]

#### Create

In [26]:
paytype_list = [
    "Basisgehalt",
    "AG-Anteil",
    "Lohnsteuer",
    "Soli-Zuschlag",
    "Krankenk.",
    "Pflegevers.",
    "Arbeitslosenvers.",
    "Rentenvers."
]

paytype_factor = [1.] + [(len(paytype_list) - i) * 0.025 for i in range(1, len(paytype_list))]

df_paytype = pd.DataFrame({
    "id": [i for i in range(1, len(paytype_list) + 1)],
    "name": paytype_list,
    "salary_factor": paytype_factor
})

tbl_db_datev_paytype.save(df_paytype)
tbl_db_datev_paytype.read()

[2023-12-21 22:59:51] TABLE successfully saved at: ./datev.dbo.pay_type.csv   SHAPE: (8, 3)


,id,name,salary_factor
0,1,Basisgehalt,1.000
1,2,AG-Anteil,0.175
2,3,Lohnsteuer,0.150
3,4,Soli-Zuschlag,0.125
4,5,Krankenk.,0.100
5,6,Pflegevers.,0.075
6,7,Arbeitslosenvers.,0.050
7,8,Rentenvers.,0.025


### dbo.employee_pay [Transaction Table]

#### Create Big Tansaction Table

In [27]:

def run_employee_pay_transations(loops: int = 1):
    transaction_date = get_next_transaction_datetime(tbl_db_datev_employee_pay)
    df_costcenter = tbl_db_datev_costcenter.read()
    df_costcenter = df_costcenter[["id", "client_id","department_id"]] \
                        .rename(columns={"id": "costcenter_id"})

    df_employee = tbl_db_datev_employee.read()

    df_employee_pay = pd.merge(
        df_employee,
        df_costcenter,
        on=["client_id", "costcenter_id"],
        how="left"
    )

    df_employee_pay = pd_crossjoin(
        df=df_employee_pay,
        join_table=tbl_db_datev_paytype,
        id_column_name_jt="id",
        id_new_column_name="paytype_id",
        include_columns=["salary_factor"]
    )


    df_employee_pay["amount"] = np.round(
        df_employee_pay["salary"] * df_employee_pay["salary_factor"],
        2
    )
    df_employee_pay[TRANSACTION_DATE_COLUMN] = transaction_date
    df_employee_pay = df_employee_pay.rename(columns={"id": "employee_id"})

    df_employee_pay = df_employee_pay[[
        "transaction_date",
        "employee_id",
        "client_id",
        "costcenter_id",
        "paytype_id",
        "amount"
    ]]

    df_employee_pay = generate_wide_df(df_employee_pay)

    for _ in range(0, loops):
        save_mode = "a" if tbl_db_datev_employee_pay.is_existing else "w"
        tbl_db_datev_employee_pay.save(df_employee_pay, mode=save_mode)
        transaction_date = get_end_of_month(transaction_date, offset_months=1)
        df_employee_pay[TRANSACTION_DATE_COLUMN] = transaction_date

if UPDATE_TRANSACTION_TABLES_THIS_RUN:
    run_employee_pay_transations(loops=1)

[2023-12-21 23:00:19] TABLE successfully saved at: ./datev.dbo.employee_pay.csv   SHAPE: (240136, 156)


In [28]:
df = tbl_db_datev_employee_pay.read(columns=[TRANSACTION_DATE_COLUMN])
print(df.shape)
list(df[TRANSACTION_DATE_COLUMN].unique())

(240136, 1)


['2000-01-31']